# 注意力机制与注意力分数
注意力是稀缺的，而环境中的干扰注意力的信息却并不少。比如我们的视觉神经系统大约每秒收到$10^8$位的信息，这远远超过了大脑能够完全处理的水平。幸运的是，我们的祖先已经从经验（也称为数据）中认识到
“并非感官的所有输入都是一样的”。在整个人类历史中，这种只将注意力引向感兴趣的一小部分信息的能力，使我们的大脑能够更明智地分配资源来生存、成长和社交，例如发现天敌、找寻食物和伴侣。

## 自主性和非自主性注意力提示
注意力是如何应用于视觉世界中的呢？我们从当今十分普及的*双组件*（two-component）的框架开始讲起：这个框架的出现可以追溯到19世纪90年代的威廉·詹姆斯，他被认为是“美国心理学之父” :cite:`James.2007`。在这个框架中，受试者基于*非自主性提示*和*自主性提示*有选择地引导注意力的焦点。

非自主性提示是基于环境中物体的突出性和易见性。想象一下，假如你面前有五个物品：一份报纸、一篇研究论文、一杯咖啡、一本笔记本和一本书，就像 :numref:`fig_eye-coffee`。所有纸制品都是黑白印刷的，但咖啡杯是红色的。换句话说，这个咖啡杯在这种视觉环境中是突出和显眼的，不由自主地引起人们的注意。所以你把视力最敏锐的地方放到咖啡上，如 :numref:`fig_eye-coffee`所示。

![由于突出性的非自主性提示（红杯子），注意力不自主地指向了咖啡杯](./eye-coffee.svg)
:width:`400px`

:label:`fig_eye-coffee`

喝咖啡后，你会变得兴奋并想读书。所以你转过头，重新聚焦你的眼睛，然后看看书，就像 :numref:`fig_eye-book`中描述那样。与 :numref:`fig_eye-coffee`中由于突出性导致的选择不同，此时选择书是受到了认知和意识的控制，因此注意力在基于自主性提示去辅助选择时将更为谨慎。受试者的主观意愿推动，选择的力量也就更强大。

![依赖于任务的意志提示（想读一本书），注意力被自主引导到书上](./eye-book.svg)
:width:`400px`

:label:`fig_eye-book`

## 查询、键和值
自主性和非自主性的注意力提示解释了人类的注意力方式，下面我们看看如何通过这两种注意力提示，用神经网络来设计注意力机制的框架。

在注意力机制的背景下，我们将自主性提示称为*查询*（query）。给定任何查询，注意力机制通过*注意力汇聚*（attention pooling）将选择引导至*感官输入*（sensory inputs，例如中间特征表示）。在注意力机制中，这些感官输入被称为*值*（value）。更通俗的解释，每个值都与一个*键*（key）配对，这可以想象为感官输入的非自主提示。如 :numref:`fig_qkv`所示，我们可以设计注意力汇聚，以便给定的查询（自主性提示）可以与键（非自主性提示）进行匹配，这将引导得出最匹配的值（感官输入）。

![注意力机制通过注意力汇聚将*查询*（自主性提示）和*键*（非自主性提示）结合在一起，实现对*值*（感官输入）的选择倾向](./qkv.svg)

:label:`fig_qkv`

## 注意力汇聚
上节我们介绍了框架下的注意力机制的主要成分 :numref:`fig_qkv`：查询（自主提示）和键（非自主提示）之间的交互形成了注意力汇聚，注意力汇聚有选择地聚合了值（感官输入）以生成最终的输出。在本节中，我们将介绍注意力汇聚的更多细节，以便从宏观上了解注意力机制在实践中的运作方式。具体来说，1964年提出的Nadaraya-Watson核回归模型是一个简单但完整的例子，可以用于演示具有注意力机制的机器学习。

考虑下面这个回归问题：给定的成对的“输入－输出”数据集
$\{(x_1, y_1), \ldots, (x_n, y_n)\}$，如何学习$f$来预测任意新输入$x$的输出$\hat{y} = f(x)$？

### 平均汇聚
我们先使用最简单的估计器来解决回归问题：基于平均汇聚来计算所有训练样本输出值的平均值：

$$f(x) = \frac{1}{n}\sum_{i=1}^n y_i,$$
:eqlabel:`eq_avg-pooling`

但很明显，这个评估器的性能会很差。

### 非参数注意力汇聚
显然，平均汇聚忽略了输入$x_i$。于是Nadaraya :cite:`Nadaraya.1964`和Watson :cite:`Watson.1964`提出了一个更好的想法，根据输入的位置对输出$y_i$进行加权：

$$f(x) = \sum_{i=1}^n \frac{K(x - x_i)}{\sum_{j=1}^n K(x - x_j)} y_i,$$
:eqlabel:`eq_nadaraya-watson`

其中$K$是*核*（kernel）。公式 :eqref:`eq_nadaraya-watson`所描述的估计器被称为*Nadaraya-Watson核回归*（Nadaraya-Watson kernel regression）。


受此启发，我们可以从 :numref:`fig_qkv`中的注意力机制框架的角度
重写 :eqref:`eq_nadaraya-watson`，成为一个更加通用的*注意力汇聚*（attention pooling）公式：

$$f(x) = \sum_{i=1}^n \alpha(x, x_i) y_i,$$
:eqlabel:`eq_attn-pooling`

其中$x$是查询，$(x_i, y_i)$是键值对。比较 :eqref:`eq_attn-pooling`和 :eqref:`eq_avg-pooling`，注意力汇聚是$y_i$的加权平均。将查询$x$和键$x_i$之间的关系建模为*注意力权重*（attention weight）$\alpha(x, x_i)$，如 :eqref:`eq_attn-pooling`所示，这个权重将被分配给每一个对应值$y_i$。对于任何查询，模型在所有键值对注意力权重都是一个有效的概率分布：它们是非负的，并且总和为1。

为了更好地理解注意力汇聚，我们考虑一个*高斯核*（Gaussian kernel），其定义为：

$$K(u) = \frac{1}{\sqrt{2\pi}} \exp(-\frac{u^2}{2}).$$

将高斯核代入 :eqref:`eq_attn-pooling`和 :eqref:`eq_nadaraya-watson`可以得到：

$$\begin{aligned} f(x) &=\sum_{i=1}^n \alpha(x, x_i) y_i\\ &= \sum_{i=1}^n \frac{\exp\left(-\frac{1}{2}(x - x_i)^2\right)}{\sum_{j=1}^n \exp\left(-\frac{1}{2}(x - x_j)^2\right)} y_i \\&= \sum_{i=1}^n \mathrm{softmax}\left(-\frac{1}{2}(x - x_i)^2\right) y_i. \end{aligned}$$
:eqlabel:`eq_nadaraya-watson-gaussian`

在 :eqref:`eq_nadaraya-watson-gaussian`中，如果一个键$x_i$越是接近给定的查询$x$，那么分配给这个键对应值$y_i$的注意力权重就会越大，也就“获得了更多的注意力”。

值得注意的是，Nadaraya-Watson核回归是一个非参数模型。因此， :eqref:`eq_nadaraya-watson-gaussian`是*非参数的注意力汇聚*（nonparametric attention pooling）模型。

### 带参数注意力汇聚
非参数的Nadaraya-Watson核回归具有*一致性*（consistency）的优点：如果有足够的数据，此模型会收敛到最优结果。尽管如此，我们还是可以轻松地将可学习的参数集成到注意力汇聚中。

例如，与 :eqref:`eq_nadaraya-watson-gaussian`略有不同，在下面的查询$x$和键$x_i$之间的距离乘以可学习参数$w$：

$$\begin{aligned}f(x) &= \sum_{i=1}^n \alpha(x, x_i) y_i \\&= \sum_{i=1}^n \frac{\exp\left(-\frac{1}{2}((x - x_i)w)^2\right)}{\sum_{j=1}^n \exp\left(-\frac{1}{2}((x - x_j)w)^2\right)} y_i \\&= \sum_{i=1}^n \mathrm{softmax}\left(-\frac{1}{2}((x - x_i)w)^2\right) y_i.\end{aligned}$$
:eqlabel:`eq_nadaraya-watson-gaussian-para`


## 注意力分数
在watson核回归中，我们使用高斯核来对查询和键之间的关系进行了建模，现在我们将高斯核指数部分视为注意力评分函数，把这个函数的输出结果输入到softmax函数中进行运算。通过上述步骤，我们将得到与键对应的值的概率分布（即注意力权重）。最后，注意力汇聚的输出就是基于这些注意力权重的值的加权和。

下图说明了如何将注意力汇聚的输出计算成为值的加权和，其中中$a$表示注意力评分函数。由于注意力权重是概率分布，因此加权和其本质上是加权平均值。
![计算注意力汇聚的输出为值的加权和](./attention-output.svg)

:label:`fig_attention_output`

用数学语言描述，假设有一个查询$\mathbf{q} \in \mathbb{R}^q$和
$m$个“键－值”对$(\mathbf{k}_1, \mathbf{v}_1), \ldots, (\mathbf{k}_m, \mathbf{v}_m)$，其中$\mathbf{k}_i \in \mathbb{R}^k$，$\mathbf{v}_i \in \mathbb{R}^v$。注意力汇聚函数$f$就被表示成值的加权和：

$$f(\mathbf{q}, (\mathbf{k}_1, \mathbf{v}_1), \ldots, (\mathbf{k}_m, \mathbf{v}_m)) = \sum_{i=1}^m \alpha(\mathbf{q}, \mathbf{k}_i) \mathbf{v}_i \in \mathbb{R}^v,$$

:eqlabel:`eq_attn-pooling`

其中查询$\mathbf{q}$和键$\mathbf{k}_i$的注意力权重（标量）是通过注意力评分函数$a$ 将两个向量映射成标量，再经过softmax运算得到的：

$$\alpha(\mathbf{q}, \mathbf{k}_i) = \mathrm{softmax}(a(\mathbf{q}, \mathbf{k}_i)) = \frac{\exp(a(\mathbf{q}, \mathbf{k}_i))}{\sum_{j=1}^m \exp(a(\mathbf{q}, \mathbf{k}_j))} \in \mathbb{R}.$$

:eqlabel:`eq_attn-scoring-alpha`

正如我们所看到的，选择不同的注意力评分函数$a$会导致不同的注意力汇聚操作。在本节中，我们将介绍两个流行的评分函数，稍后将用他们来实现更复杂的注意力机制。

### 掩蔽softmax操作
正如上面提到的，softmax操作用于输出一个概率分布作为注意力权重。
在某些情况下，并非所有的值都应该被纳入到注意力汇聚中。例如，为了在机器翻译中高效处理小批量数据集，某些文本序列被填充了没有意义的特殊词元。为了仅将有意义的词元作为值来获取注意力汇聚，我们可以指定一个有效序列长度（即词元的个数），以便在计算softmax时过滤掉超出指定范围的位置。通过这种方式，我们可以在下面的`masked_softmax`函数中实现这样的*掩蔽softmax操作*（masked softmax operation），其中任何超出有效长度的位置都被掩蔽并置为0。

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        return nn.functional.softmax(X.reshape(shape), dim=-1)

为了[**演示此函数是如何工作**]的，考虑由两个$2 \times 4$矩阵表示的样本，这两个样本的有效长度分别为$2$和$3$。经过掩蔽softmax操作，超出有效长度的值都被掩蔽为0。

In [17]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.6578, 0.3422, 0.0000, 0.0000],
         [0.4089, 0.5911, 0.0000, 0.0000]],

        [[0.3648, 0.4210, 0.2142, 0.0000],
         [0.3205, 0.2670, 0.4124, 0.0000]]])

同样，我们也可以使用二维张量，为矩阵样本中的每一行指定有效长度。


In [18]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3792, 0.3491, 0.2717, 0.0000]],

        [[0.4437, 0.5563, 0.0000, 0.0000],
         [0.1742, 0.3892, 0.2275, 0.2090]]])

### 加性注意力
一般来说，当查询和键是不同长度的矢量时，我们可以使用加性注意力作为评分函数。给定查询$\mathbf{q} \in \mathbb{R}^q$和键$\mathbf{k} \in \mathbb{R}^k$，*加性注意力*（additive attention）的评分函数为

$$a(\mathbf q, \mathbf k) = \mathbf w_v^\top \text{tanh}(\mathbf W_q\mathbf q + \mathbf W_k \mathbf k) \in \mathbb{R},$$
:eqlabel:`eq_additive-attn`

其中可学习的参数是$\mathbf W_q\in\mathbb R^{h\times q}$、$\mathbf W_k\in\mathbb R^{h\times k}$和$\mathbf w_v\in\mathbb R^{h}$。如 :eqref:`eq_additive-attn`所示，将查询和键连结起来后输入到一个多层感知机（MLP）中，感知机包含一个隐藏层，其隐藏单元数是一个超参数$h$。通过使用$\tanh$作为激活函数，并且禁用偏置项。

下面我们来实现加性注意力。

In [26]:
class AdditiveAttention(nn.Module):
    """加性注意力"""
    def __init__(self, key_size, query_size, num_hiddens, dropout):
        super(AdditiveAttention, self).__init__()
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, queries, keys, values, valid_lens):
        # 输入queries: （batch_size, num_queries, qurey_size）
        # 输入keys：（batch_size, num_keys, key_size）
        # 输入values:(batch_size, num_value, value_size)
        queries, keys = self.W_q(queries), self.W_k(keys)
        # 输出quries：（batch_size, num_quries, num_hiddens）
        # 输出keys：（batch_size, num_keys, num_hiddens）
        # 在维度扩展后，
        # queries的形状：(batch_size，查询的个数，1，num_hiddens)
        # key的形状：(batch_size，1，“键－值”对的个数，num_hiddens)
        # 使用广播方式进行求和
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        features = torch.tanh(features)
        # self.w_v仅有一个输出，因此从形状中移除最后那个维度。
        # scores的形状：(batch_size，查询的个数，“键-值”对的个数)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        # values的形状：(batch_size，“键－值”对的个数，值的维度)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [57]:
queries, keys = torch.normal(0, 1, (2, 1, 20)), torch.ones((2, 10, 2))
# values的小批量，两个值矩阵是相同的
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(
    2, 1, 1)
valid_lens = torch.tensor([2, 6])

In [58]:
values.shape

torch.Size([2, 10, 4])

In [28]:
attention = AdditiveAttention(key_size=2, query_size=20, num_hiddens=8,
                              dropout=0.1)

In [29]:
attention.eval()
attention(queries, keys, values, valid_lens)

tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]], grad_fn=<BmmBackward0>)

### 缩放点积注意力
使用点积可以得到计算效率更高的评分函数，但是点积操作要求查询和键具有相同的长度$d$。假设查询和键的所有元素都是独立的随机变量，
并且都满足零均值和单位方差，那么两个向量的点积的均值为$0$，方差为$d$。为确保无论向量长度如何，点积的方差在不考虑向量长度的情况下仍然是$1$，我们将点积除以$\sqrt{d}$，则*缩放点积注意力*（scaled dot-product attention）评分函数为：

$$a(\mathbf q, \mathbf k) = \mathbf{q}^\top \mathbf{k}  /\sqrt{d}.$$

在实践中，我们通常从小批量的角度来考虑提高效率，例如基于$n$个查询和$m$个键－值对计算注意力，其中查询和键的长度为$d$，值的长度为$v$。查询$\mathbf Q\in\mathbb R^{n\times d}$、键$\mathbf K\in\mathbb R^{m\times d}$和值$\mathbf V\in\mathbb R^{m\times v}$的缩放点积注意力是：

$$ \mathrm{softmax}\left(\frac{\mathbf Q \mathbf K^\top }{\sqrt{d}}\right) \mathbf V \in \mathbb{R}^{n\times v}.$$
:eqlabel:`eq_softmax_QK_V`

在下面的缩放点积注意力的实现中，我们使用了暂退法进行模型正则化。

In [30]:
class DotProductAttention(nn.Module):
    """缩放点积注意力"""
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        # 设置transpose_b=True为了交换keys的最后两个维度
        scores = torch.bmm(queries, keys.transpose(1,2)) / math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)

为了[**演示上述的`DotProductAttention`类**]，我们使用与先前加性注意力例子中相同的键、值和有效长度。对于点积操作，我们令查询的特征维度与键的特征维度大小相同。

In [31]:
queries = torch.normal(0, 1, (2, 1, 2))
attention = DotProductAttention(dropout=0.5)
attention.eval()
attention(queries, keys, values, valid_lens)

tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]])

In [38]:
import torch
queries = torch.normal(0, 1, (2, 1, 8))
keys = torch.normal(0, 1, (2, 3, 8))

In [39]:
queries, keys

(tensor([[[ 1.1320, -1.9323,  0.2179,  0.5782,  1.0536, -0.1332, -0.4476,
            0.8335]],
 
         [[-0.2599,  1.0570, -0.8896, -0.9959,  0.5764, -0.3724,  0.3278,
           -1.4470]]]),
 tensor([[[ 0.4880, -0.2467, -0.4451,  1.5075,  1.2238, -0.5534,  1.5106,
           -1.0668],
          [-2.0621,  0.5645, -0.2842,  1.5053,  0.2680, -1.0845, -1.6279,
           -2.3402],
          [-1.0910, -0.2694, -0.7549,  2.5518,  0.0648,  0.3785,  0.0258,
            0.7397]],
 
         [[ 1.1851,  0.3699,  1.0238, -0.3646, -0.0181, -1.1888, -0.8078,
           -0.0510],
          [ 1.3109,  0.8196, -0.6464,  0.5376,  1.0462, -0.1900,  0.1942,
            2.0457],
          [-1.9543, -1.3421,  0.2868,  0.7844,  0.7489, -0.7335,  0.5136,
           -0.8876]]]))

In [41]:
queries.unsqueeze(2).shape

torch.Size([2, 1, 1, 8])

In [42]:
keys.unsqueeze(1).shape

torch.Size([2, 1, 3, 8])

In [44]:
(queries.unsqueeze(2) + keys.unsqueeze(1)).shape

torch.Size([2, 1, 3, 8])

In [45]:
w_v = torch.nn.Linear(8, 1, bias=False)

In [48]:
w_v((queries.unsqueeze(2) + keys.unsqueeze(1))).shape

torch.Size([2, 1, 3, 1])

In [49]:
w_v((queries.unsqueeze(2) + keys.unsqueeze(1)))

tensor([[[[ 0.1808],
          [ 0.0310],
          [ 0.9627]]],


        [[[-0.9054],
          [-0.4660],
          [-0.8626]]]], grad_fn=<UnsafeViewBackward0>)

In [50]:
w_v((queries.unsqueeze(2) + keys.unsqueeze(1))).squeeze(-1)

tensor([[[ 0.1808,  0.0310,  0.9627]],

        [[-0.9054, -0.4660, -0.8626]]], grad_fn=<SqueezeBackward1>)

In [51]:
w_v((queries.unsqueeze(2) + keys.unsqueeze(1))).squeeze(-1).shape

torch.Size([2, 1, 3])

In [52]:
masked_softmax(w_v((queries.unsqueeze(2) + keys.unsqueeze(1))).squeeze(-1), torch.tensor([1, 2]))

tensor([[[1.0000, 0.0000, 0.0000]],

        [[0.3919, 0.6081, 0.0000]]], grad_fn=<SoftmaxBackward0>)

In [53]:
masked_softmax(w_v((queries.unsqueeze(2) + keys.unsqueeze(1))).squeeze(-1), torch.tensor([1, 2])).shape

torch.Size([2, 1, 3])

In [54]:
values = torch.normal(0, 0.1, (2, 3, 8))

In [55]:
torch.bmm(masked_softmax(w_v((queries.unsqueeze(2) + keys.unsqueeze(1))).squeeze(-1), torch.tensor([1, 2])), values)

tensor([[[-0.1643, -0.2285,  0.0530,  0.1367, -0.0682,  0.1698,  0.0310,
           0.0453]],

        [[ 0.0577, -0.1506, -0.0532,  0.0370,  0.0350,  0.0443,  0.1071,
          -0.1172]]], grad_fn=<BmmBackward0>)

In [56]:
torch.bmm(masked_softmax(w_v((queries.unsqueeze(2) + keys.unsqueeze(1))).squeeze(-1), torch.tensor([1, 2])), values).shape

torch.Size([2, 1, 8])